# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.10.57 openai==1.59.8 cohere==5.6.2 tiktoken==0.7.0 chromadb==0.5.5 html2text==2024.2.26 sentence-transformers==3.3.1 pydantic==2.10.5 llama-index-vector-stores-chroma==0.1.10 kaleido==0.2.1 llama-index-llms-gemini==0.1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR_GOOGLE_API_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

In [4]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.
import nest_asyncio

nest_asyncio.apply()

# Load a Model


In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


In [6]:
# Downloading Vector store from Hugging face hub
from huggingface_hub import hf_hub_download

vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip", repo_type="dataset", local_dir=".")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [7]:
!unzip vectorstore.zip

Archive:  vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [8]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

# Create your index
db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
vector_index = VectorStoreIndex.from_vector_store(vector_store)

In [9]:
# Query Engine
ai_tutor_knowledge_query_engine = vector_index.as_query_engine(similarity_top_k=3)

res = ai_tutor_knowledge_query_engine.query("How does Retrieval Augmented Generation (RAG) work?")
print(res.response)

Retrieval Augmented Generation (RAG) operates through a systematic workflow that integrates two primary components: Retrieval and Generation. The process begins with the Retrieval component, which extracts relevant information from external knowledge sources. This involves two main phases: indexing, where documents are organized for efficient access, and searching, which retrieves relevant documents based on user queries. Often, rerankers are used to refine the document ranking during this phase.

Following retrieval, the Generation component utilizes the retrieved content along with user queries to generate coherent and contextually relevant responses. It employs various prompting techniques, such as Chain of Thought and Tree of Thought, to enhance the response quality. During the inferencing phase, the model interprets the prompted input to produce accurate answers that align with the intent of the query, integrating the extracted information without requiring additional fine-tuning.

In [10]:
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("Metadata\t", src.metadata)
    print("-_" * 20)

Node ID	 2aa05360-f43a-4819-bce7-0acf7b897eab
Title	 Searching for Best Practices in Retrieval-Augmented Generation:1 Introduction
Text	 Generative large language models are prone to producing outdated information or fabricating facts, although they were aligned with human preferences by reinforcement learning [1] or lightweight alternatives [2–5]. Retrieval-augmented generation (RAG) techniques address these issues by combining the strengths of pretraining and retrieval-based models, thereby providing a robust framework for enhancing model performance [6]. Furthermore, RAG enables rapid deployment of applications for specific organizations and domains without necessitating updates to the model parameters, as long as query-related documents are provided. Many RAG approaches have been proposed to enhance large language models (LLMs) through query-dependent retrievals [6–8]. A typical RAG workflow usually contains multiple intervening processing steps: query classification (determining w

# Router

Routers are modules that take in a user query and a set of “choices” (defined by metadata), and returns one or more selected choices.

They can be used for the following use cases and more:

- Selecting the right data source among a diverse range of data sources

- Deciding whether to do summarization (e.g. using summary index query engine) or semantic search (e.g. using vector index query engine)

- Deciding whether to “try” out a bunch of choices at once and combine the results (using multi-routing capabilities).


## Lets create a different query engine with Mistral AI information


In [11]:
from pathlib import Path
import requests

wiki_titles = [
    "Mistral AI",
    "Llama (language model)",
    "Claude AI",
    "OpenAI",
    "Gemini AI",
]

data_path = Path("llm_data_wiki")

if not data_path.exists():
    data_path.mkdir()

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()

    page = next(iter(response["query"]["pages"].values()))

    if "extract" in page:
        wiki_text = page["extract"]

        with open(data_path / "llm_data_wiki.txt", "a") as fp:
            fp.write(f"Title: {title}\n{wiki_text}\n\n")
    else:
        print(f"No extract found for '{title}'")


In [12]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.text_splitter import TokenTextSplitter
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)

# Assuming you have prepared a directory for llm data
documents = SimpleDirectoryReader("llm_data_wiki").load_data()

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

transformations = [
    text_splitter,
    QuestionsAnsweredExtractor(questions=2),
    SummaryExtractor(summaries=["prev", "self"]),
    KeywordExtractor(keywords=10),
    OpenAIEmbedding(model="text-embedding-3-small"),
]

llm_index = VectorStoreIndex.from_documents(documents=documents, transformations=transformations)

llm_query_engine = llm_index.as_query_engine(similarity_top_k=2)

100%|██████████| 50/50 [01:01<00:00,  1.23s/it]


In [13]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool

# initialize tools
ai_tutor_knowledge_tool = QueryEngineTool.from_defaults(
    query_engine=ai_tutor_knowledge_query_engine,
    description="Useful for questions about general generative AI concepts",
)
llm_tool = QueryEngineTool.from_defaults(
    query_engine=llm_query_engine,
    description="Useful for questions about particular LLMs like Mistral, Claude, OpenAI, Gemini",
)

# initialize router query engine (single selection, pydantic)
query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        ai_tutor_knowledge_tool,
        llm_tool,
    ],
)

In [14]:
res = query_engine.query(
    "What is the LLama model?",
)
print(res.response)

The Llama model is a series of language models developed by Meta Platforms. It has undergone several iterations, starting with the original LLaMA model, and subsequent versions include LLaMa 2 and Llama 3. These models are designed for various applications in artificial intelligence, leveraging large datasets for training to improve their performance on numerous benchmarks. The architecture of these models has remained largely consistent, but improvements have been made in terms of training data volume, with LLaMa 2 trained on 40% more data than its predecessor, and Llama 3 trained on approximately 15 trillion tokens from publicly available sources. Llama models are also characterized by the adoption of open weights, allowing for broader developer adoption while raising concerns regarding potential misuse.


In [15]:
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 956e4e0b-feac-45cf-9f3c-3778ec9cd005
Text	 format focuses on supporting different quantization types, which can reduce memory usage, and increase speed at the expense of lower model precision.
llamafile created by Justine Tunney is an open-source tool that bundles llama.cpp with the model into a single executable file. Tunney et al. introduced new optimized matrix multiplication kernels for x86 and ARM CPUs, improving prompt evaluation performance for FP16 and 8-bit quantized data types.


=== Military ===
In 2024, researchers from the People's Liberation Army Academy of Military Sciences (top military academy of China) were reported to have developed a military tool using Llama, which Meta Platforms stated was unauthorized due to Llama's license prohibiting the use of the model for military purposes. Meta granted the US government and US military contractors permission to use Llama in November 2024, but continued to prohibit military use by non-US entities.


== Reception ==


In [16]:
res = query_engine.query("Explain parameter-efficient finetuning methods")
print(res.response)

Parameter-efficient fine-tuning methods are designed to optimize the training of large language models (LLMs) by minimizing the computational resources required and the number of trainable parameters. Three main approaches are commonly used in these methods:

1. **Selective Fine-Tuning**: This approach involves fine-tuning only a subset of the parameters in the model, rather than all parameters. This selective strategy results in reduced computational costs and retains the majority of the pre-trained model's knowledge.

2. **Reparameterization**: This method uses low-rank representations to adjust the model's weights. A well-known technique under this category is Low Rank Adaptation (LoRA), which reduces the number of trainable parameters by approximating the original weight matrix with two smaller matrices. For example, a weight matrix of size 512 x 64 can be approximated using matrices of lower rank, drastically reducing the number of parameters needed for fine-tuning.

3. **Additive

In [17]:
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 6be88fa3-2f8b-43e7-aba0-d874b39809fc
Text	 # FourierFT: Discrete Fourier Transformation Fine-Tuning[FourierFT](https://huggingface.co/papers/2405.03003) is a parameter-efficient fine-tuning technique that leverages Discrete Fourier Transform to compress the model's tunable weights. This method outperforms LoRA in the GLUE benchmark and common ViT classification tasks using much less parameters.FourierFT currently has the following constraints:- Only `nn.Linear` layers are supported.- Quantized layers are not supported.If these constraints don't work for your use case, consider other methods instead.The abstract from the paper is:> Low-rank adaptation (LoRA) has recently gained much interest in fine-tuning foundation models. It effectively reduces the number of trainable parameters by incorporating low-rank matrices A and B to represent the weight change, i.e., Delta W=BA. Despite LoRA's progress, it faces storage challenges when handling extensive customization adaptations or 

# OpenAI Agent

In [18]:
from llama_index.agent.openai import OpenAIAgent

In [19]:
system_message_openai_agent = """You are an AI teacher, answering questions from students of an applied AI course on Large Language Models (LLMs or llm) and Retrieval Augmented Generation (RAG) for LLMs. Topics covered include training models, fine-tuning models, giving memory to LLMs, prompting tips, hallucinations and bias, vector databases, transformer architectures, embeddings, RAG frameworks, Langchain, LlamaIndex, making LLMs interact with tools, AI agents, reinforcement learning with human feedback. Questions should be understood in this context.

Your answers are aimed to teach students, so they should be complete, clear, and easy to understand.

Use the available tools to gather insights pertinent to the field of AI. Always use two tools at the same time. These tools accept a string (a user query rewritten as a statement) and return informative content regarding the domain of AI.
e.g:
User question: 'How can I fine-tune an LLM?'
Input to the tool: 'Fine-tuning an LLM'

User question: How can quantize an LLM?
Input to the tool: 'Quantization for LLMs'

User question: 'Teach me how to build an AI agent"'
Input to the tool: 'Building an AI Agent'

Only some information returned by the tools might be relevant to the question, so ignore the irrelevant part and answer the question with what you have.

Your responses are exclusively based on the output provided by the tools. Refrain from incorporating information not directly obtained from the tool's responses.

When the conversation deepens or shifts focus within a topic, adapt your input to the tools to reflect these nuances. This means if a user requests further elaboration on a specific aspect of a previously discussed topic, you should reformulate your input to the tool to capture this new angle or more profound layer of inquiry.

Provide comprehensive answers, ideally structured in multiple paragraphs, drawing from the tool's variety of relevant details. The depth and breadth of your responses should align with the scope and specificity of the information retrieved.

Should the tools repository lack information on the queried topic, politely inform the user that the question transcends the bounds of your current knowledge base, citing the absence of relevant content in the tool's documentation.

At the end of your answers, always invite the students to ask deeper questions about the topic if they have any. Make sure to reformulate the question to the tool to capture this new angle or more profound layer of inquiry.

Do not refer to the documentation directly, but use the information provided within it to answer questions.

If code is provided in the information, share it with the students. It's important to provide complete code blocks so they can execute the code when they copy and paste them.

Make sure to format your answers in Markdown format, including code blocks and snippets.

Politely reject questions not related to AI, while being cautious not to reject unfamiliar terms or acronyms too quickly."""

In [20]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

agent = OpenAIAgent.from_tools(
    llm=llm,
    tools=[ai_tutor_knowledge_tool, llm_tool],
    system_prompt=system_message_openai_agent,
)

In [21]:
response = agent.chat("What is the LLama model?")
print(response.response)

The LLaMA (Large Language Model Meta AI) series is a collection of language models developed by Meta, showcasing significant advancements in AI language processing. Here's an overview of the different iterations of the LLaMA models:

1. **LLaMA**: The original model was announced on February 24, 2023. It was notable for its efficiency, with the 13 billion parameter version outperforming larger models like GPT-3, which has 175 billion parameters. This demonstrated that smaller models could achieve competitive results with well-structured training. However, the model's weights were leaked online shortly after its release, leading to legal actions by Meta to control unauthorized distribution.

2. **LLaMa 2**: Released on July 18, 2023, in collaboration with Microsoft, LLaMa 2 introduced three sizes: 7 billion (B), 13B, and 70B parameters. This version was trained on 40% more data than its predecessor and was made available for commercial use under an acceptable use policy. The architectur

In [22]:
response = agent.chat("Explain parameter-efficient finetuning methods")
print(response.response)

Parameter-efficient fine-tuning methods are designed to enhance the performance of large language models (LLMs) without requiring extensive computational resources or modifications to the entire model. Here are some key techniques used in this context:

1. **Autoregressive Loss Function**: This method involves fine-tuning models using an autoregressive loss function, which helps in predicting the next token in a sequence based on the previous ones. This approach is efficient as it leverages the existing structure of the model to improve its performance on specific tasks.

2. **Batch Size and Context Length**: Fine-tuning often involves using a specific batch size and context length to optimize the training process. For instance, a batch size of 64 and a context length of 4K tokens are used to ensure that the model can handle large inputs effectively while maintaining computational efficiency.

3. **Reinforcement Learning from Human Feedback (RLHF)**: This technique involves using human

In [23]:
response = agent.chat("Write the recipe for a chocolate cake.")
print(response.response)

I'm here to assist with questions related to AI, particularly in the context of large language models and related technologies. If you have any questions about AI, feel free to ask!


# Code related questions to GPT-4o, the remaining questions to Gemini

In [24]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.llms.gemini import Gemini
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool

# initialize LLMs
gpt_4o_llm = OpenAI(model="gpt-4o")
gemini_llm = Gemini(model="models/gemini-1.5-flash", temperature=1, max_tokens=512)

# define query engines
llm_query_engine_code = vector_index.as_query_engine(
    llm=gpt_4o_llm,
    similarity_top_k=3,
    embed_model=OpenAIEmbedding(model="text-embedding-3-small", mode="text_search"),
)

llm_query_engine_rest = vector_index.as_query_engine(
    llm=gemini_llm,
    similarity_top_k=3,
    embed_model=OpenAIEmbedding(model="text-embedding-3-small", mode="text_search"),
)

# define tools for LLM
llm_tool_code = QueryEngineTool.from_defaults(
    query_engine=llm_query_engine_code,
    description="Ideal for handling code-related queries, technical implementations, and troubleshooting involving Large Language Models.",
    name="LLMCodeTool",
)

llm_tool_rest = QueryEngineTool.from_defaults(
    query_engine=llm_query_engine_rest,
    description="Best suited for answering conceptual, theoretical, and general questions about Large Language Models.",
    name="LLMGeneralTool",
)

# Initialize OpenAIAgent with the system message and the router query engine
agent = OpenAIAgent.from_tools(
    llm=gpt_4o_llm,  # The base LLM, used only if no other tools apply
    tools=[llm_tool_code, llm_tool_rest],
    system_prompt=system_message_openai_agent,
)

In [25]:
# Test the agent with a code-related question
response = agent.chat("How do I fine-tune the LLama model? Write the code for it.")
for source in response.sources:
    print(source.tool_name)

LLMGeneralTool
LLMCodeTool


In [29]:
# Test the agent with a Non code-related question
response1 = agent.chat("What is the relationship between Llama models and Meta?")
for source in response1.sources:
    print(source.tool_name)

LLMGeneralTool
